In [40]:
include("DCG.jl")
include("cluster_test_helper.jl");

In [21]:
const GRB_ENV = Gurobi.Env()

Academic license - for non-commercial use only


Gurobi.Env(Ptr{Nothing} @0x000000007d29c6f0, false, 0)

In [27]:
import Pkg;
Pkg.status("Gurobi")

      Status `C:\Users\mit\.julia\environments\v1.7\Project.toml`
  [2e9cd046] Gurobi v0.11.4


In [22]:
m = Model(() -> Gurobi.Optimizer(GRB_ENV))

@variable(m, x >= 1)
@objective(m, Min, x)
optimize!(m)

Gurobi Optimizer version 9.0.3 build v9.0.3rc0 (win64)
Optimize a model with 0 rows, 1 columns and 0 nonzeros
Model fingerprint: 0x449ace9c
Coefficient statistics:
  Matrix range     [0e+00, 0e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [0e+00, 0e+00]
Presolve removed 0 rows and 1 columns
Presolve time: 0.01s
Presolve: All rows and columns removed
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.0000000e+00   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.01 seconds
Optimal objective  1.000000000e+00

User-callback calls 19, time in user-callback 0.00 sec


In [24]:
objective_bound(m)

1.0

In [41]:
using JSON

In [33]:
date = "20221123"
number = "5"
"data/experiment_inputs/" * date * "/" * number * ".json"

"data/experiment_inputs/20221123/5.json"

In [3]:
external_params = JSON.parse(open("data/experiment_inputs/20221121/1.json"))

Dict{String, Any} with 1 entry:
  "dcg" => Dict{String, Any}("num_crews"=>10, "num_fires"=>3, "line_per_crew"=>…

In [5]:
input_params = JSON.parse(open("data/experiment_inputs/20221122/1.json"))
input_params["dcg"]

Dict{String, Any} with 3 entries:
  "num_crews"     => 10
  "num_fires"     => 3
  "line_per_crew" => 17

In [82]:
logs = []

Any[]

In [83]:
for file = 1:5
    input_params = JSON.parsefile("data/experiment_inputs/20221123/" * string(file) * ".json")
    output_params = JSON.parsefile("data/experiment_outputs/20221123/" * string(file) * ".json")

    fires = input_params["dcg"]["num_fires"]
    crews = input_params["dcg"]["num_crews"]
    cg_time = sum(values(output_params["initial_DCG"]["times"]))
    ws_time = output_params["initial_DCG"]["times"]["ws"]
    cg_time = cg_time - ws_time
    int_aware_time = sum(output_params["generate_additional_plans"]["timings"])
    restore_int_form_time = output_params["restore_integrality"]["formulation_time" ]
    restore_int_solve_time = output_params["restore_integrality"]["solve_time" ]
    mp_sol = output_params["initial_DCG"]["solutions"]["master_problem_reconstructed"]
    gap = (output_params["restore_integrality"]["pb_objective"] - mp_sol) / mp_sol 
    pb_gap = output_params["restore_integrality"]["pb_objective"] / output_params["restore_integrality"]["pb_objective_bound"] - 1 

    log = [fires, crews, ws_time, cg_time, int_aware_time, 
        restore_int_form_time, restore_int_solve_time, gap, pb_gap]
    push!(logs, log)
end

In [98]:
titles = ["fires", "crews", "t_ws", "t_dcg", "t_dcg2", "t_int_form", "t_int_solve", "gap", "int_gap"]

data = round.(1000 * reduce(hcat, logs)') / 1000;

2×1 Matrix{Array}:
 ["fires", "crews", "t_ws", "t_dcg", "t_dcg2", "t_int_form", "t_int_solve", "gap", "int_gap"]
 [3.0 10.0 … 0.052 0.0; 6.0 20.0 … 0.043 0.0; … ; 12.0 40.0 … 0.017 0.0; 15.0 50.0 … 0.071 0.059]

In [100]:
print(titles)

["fires", "crews", "t_ws", "t_dcg", "t_dcg2", "t_int_form", "t_int_solve", "gap", "int_gap"]

In [105]:
data

5×9 Matrix{Float64}:
  3.0  10.0   0.638    2.001    5.211   0.434     0.441  0.052  0.0
  6.0  20.0   4.656    8.234   33.569   3.811    19.565  0.043  0.0
  9.0  30.0   9.312   19.216  116.571  12.125   247.506  0.033  0.0
 12.0  40.0  13.51    58.403  300.228  29.857  1769.36   0.017  0.0
 15.0  50.0  29.917  100.925  557.903  50.181  3550.06   0.071  0.059

In [54]:
restore_int_form_time = output_params["restore_integrality"]["formulation_time" ]
restore_int_solve_time = output_params["restore_integrality"]["solve_time" ]

3550.06170304

In [57]:
mp_sol = output_params["initial_DCG"]["solutions"]["master_problem_reconstructed"]

6.920234610090223e6

In [61]:
gap = (output_params["restore_integrality"]["pb_objective"] - mp_sol) / mp_sol 

0.07144262036235353

In [63]:
pb_gap = output_params["restore_integrality"]["pb_objective"] / output_params["restore_integrality"]["pb_objective_bound"] - 1 

0.058966928586088097

In [66]:
[ws_time, cg_time, int_aware_time, restore_int_form_time, restore_int_solve_time, gap, pb_gap]

7-element Vector{Float64}:
   29.916918834
  100.92524869999997
  557.903429961
   50.181383677
 3550.06170304
    0.07144262036235353
    0.058966928586088097

In [37]:
output_params["initial_DCG"]

Dict{String, Any} with 6 entries:
  "iterations"        => 75
  "allotments"        => Dict{String, Any}("master_problem_reconstructed"=>Any[…
  "allotment_history" => Any[Any[Any[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.…
  "times"             => Dict{String, Any}("mp_reformulate"=>8.00982, "init_cg"…
  "solutions"         => Dict{String, Any}("master_problem_reconstructed"=>6.92…
  "rhos"              => Any[Any[Any[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.…

In [38]:
output_params["initial_DCG"]["solutions"]["master_problem_reconstructed"]

6.920234569890007e6

In [39]:
output_params["restore_integrality"]

Dict{String, Any} with 3 entries:
  "formulation_time" => 72.0157
  "pb_objective"     => 7.83762e6
  "solve_time"       => 228.228

In [26]:
GRB_ENV

LoadError: UndefVarError: GRB_ENV not defined

In [21]:
output_params["initial_DCG"]["solutions"]

Dict{String, Any} with 2 entries:
  "master_problem_reconstructed" => 1.1388e6
  "master_problem"               => 1.13868e6

In [22]:
output_params["initial_DCG"]["times"]

Dict{String, Any} with 5 entries:
  "mp_reformulate" => 0.131685
  "init_cg"        => 0.136646
  "cg"             => 1.35469
  "mp_resolve"     => 0.0043777
  "ws"             => 0.564245

In [23]:
output_params["generate_additional_plans"]

Dict{String, Any} with 2 entries:
  "iterations" => Any[23, 21, 17, 36, 37]
  "timings"    => Any[0.857676, 0.802969, 0.733287, 1.41918, 1.32183]

In [24]:
output_params["restore_integrality"]

Dict{String, Any} with 3 entries:
  "formulation_time" => 0.449316
  "pb_objective"     => 1.16727e6
  "solve_time"       => 0.459319

In [23]:
dcg_params = default_params()

for key in keys(external_params["dcg"])
    dcg_params[key] = external_params["dcg"][key]
end

In [24]:
global NUM_FIRES = dcg_params["num_fires"]
global NUM_CREWS = dcg_params["num_crews"]
global LINE_PER_CREW = dcg_params["line_per_crew"]

capacity_perturbations = [-2, -1, 0, 1, 2]

preprocessed_data = preprocess(dcg_params["in_path"])

t = @elapsed d, cg_data = single_DCG_node(dcg_params, deepcopy(preprocessed_data))
@assert dcg_params != "calculate"

dcg_params["int_aware_capacities"] = d["allotments"]["master_problem_reconstructed"]
iterations, timings, cg_data = generate_new_plans(dcg_params, cg_data, capacity_perturbations, -1, -1)
form_time, sol_time, pb = restore_integrality(cg_data, 300);

In [25]:
pb_allot = convert.(Int, round.(100 .* get_fire_allotments(pb, cg_data)) ./ 100)

3×14 Matrix{Int64}:
  0   0  0  0  1  7  9  7  8  4  3  1  0  0
  0  10  9  6  5  0  1  0  0  0  0  0  0  0
 10   0  0  0  0  0  0  0  0  0  0  0  0  0

In [26]:
form_time, sol_time

(0.4555964, 0.5811179)

In [27]:
d["solutions"], objective_value(pb["m"])

(Dict("master_problem_reconstructed" => 1.138800534771232e6, "master_problem" => 1.1386765294961422e6), 1.1672659340711462e6)

In [13]:
t

10.4605879

In [14]:
d

Dict{String, Any} with 7 entries:
  "iterations"        => 46
  "mp"                => Dict{String, Any}("sigma"=>ConstraintRef{Model, MathOp…
  "allotment_history" => Any[[0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0…
  "allotments"        => Dict{String, Any}("master_problem_reconstructed"=>[0.0…
  "times"             => Dict("mp_reformulate"=>0.80025, "init_cg"=>0.599814, "…
  "solutions"         => Dict("master_problem_reconstructed"=>2.67122e6, "maste…
  "rhos"              => Any[[0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0…

In [12]:
d

Dict{String, Any} with 7 entries:
  "iterations"        => 46
  "mp"                => Dict{String, Any}("sigma"=>ConstraintRef{Model, MathOp…
  "allotment_history" => Any[[0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0…
  "allotments"        => Dict{String, Any}("master_problem_reconstructed"=>[0.0…
  "times"             => Dict("mp_reformulate"=>0.797122, "init_cg"=>0.945321, …
  "solutions"         => Dict("master_problem_reconstructed"=>2.67122e6, "maste…
  "rhos"              => Any[[0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0…

In [261]:
d["solutions"], objective_value(pb["m"])

(Dict("master_problem_reconstructed" => 1.138800534771232e6, "master_problem" => 1.1386765294961422e6), 1.1672659340711462e6)

In [97]:
existing_plans = [[cg_data.suppression_plans.crews_present[i, j, :] 
                     for j=1:cg_data.suppression_plans.plans_per_fire[i]
                  ]
                  for i=1:NUM_FIRES 
                 ];

existing_routes = [[cg_data.routes.fires_fought[i, j, :, :] 
                     for j=1:cg_data.routes.routes_per_crew[i]
                  ]
                  for i=1:NUM_CREWS
                 ]

for excess in excesses

    pattern = [(excess, 1e30)]
    params["int_aware_price_adjustment"] = [pattern for i in 1:params["max_iters"]]

    d2, cg_data2 = single_DCG_node(params, deepcopy(preprocessed_data))


    for fire in 1:NUM_FIRES

        for plan_ix in 1:cg_data2.suppression_plans.plans_per_fire[fire] 

            if value(d2["mp"]["plan"][fire, plan_ix]) > fire_thresh

                plan = cg_data2.suppression_plans.crews_present[fire, plan_ix, :]
                cost = cg_data2.suppression_plans.plan_costs[fire, plan_ix]

                if ~(plan in existing_plans[fire])

                    push!(existing_plans[fire], plan)
                    update_available_supp_plans(fire, cost, plan, cg_data.suppression_plans)

                end
            end
        end
    end 


    for crew in 1:NUM_CREWS

        for route_ix in 1:cg_data2.routes.routes_per_crew[crew] 

            if value(d2["mp"]["route"][crew, route_ix]) > crew_thresh

                route = cg_data2.routes.fires_fought[crew, route_ix, :, :]
                cost = cg_data2.routes.route_costs[crew, route_ix]
                oo_region = cg_data2.routes.out_of_reg[crew, route_ix, :]

                if ~(route in existing_routes[crew])

                    push!(existing_routes[crew], route)
                    update_available_crew_routes(crew, cost, route, oo_region, cg_data.routes)
                end

            end
        end
    end 
end




In [154]:
verbose = true

true

In [191]:
fc = (3, 10)
    
output = Dict()

fire_thresh = -1.0001
crew_thresh = -1.0001

params = Dict{String, Any}()
params["gamma"] = 0

params["in_path"] = "data/raw/big_fire"
params["agg_prec"] = 10
params["passive_states"] = 30
params["num_fires"] = fc[1]
params["num_crews"] = fc[2]
params["line_per_crew"] = 17
params["fire_solver_type"] = "dp"

params["apply_warm_start"] = false
params["restore_cost"] = false

params["solve_explicit_lin_time_limit"] = 0
params["solve_explicit_int_time_limit"] = 0
params["cg_time_limit"] = 300
params["solve_net_flow_time_limit"] = 0

params["max_iters"] = 1000

params["dual_stab_type"] = "global"
params["dual_stab_anchor"] = false
params["dual_stab_eps"] = 0.01
params["dual_warm_start"] = "calculate"

params["ws_dual_weight"] = zeros(params["max_iters"])
params["int_aware_dual_weight"] = zeros(params["max_iters"])

pattern = [(0, 0)]
params["int_aware_price_adjustment"] = [pattern for i in 1:params["max_iters"]]


r_per_c = []
p_per_f = []

d, cg_data = run_tests(params)


params["int_aware_capacities"] = d[7]["master_problem"]

if verbose
    println("Fires : $(params["num_fires"])")
    println("Crews : $(params["num_crews"])")
    println()

    println("Running first DCG")
    println("__________")
    println("Iterations: $(d[4])")
    println("Objective values: $(d[6])")
    println("Timings: $(d[5])")
    println()

    println("Running integraity-aware DCG")
    println("__________")
end

output["intital_DCG_iterations"] = d[4]
output["intital_DCG_timing"] = d[5]
output["intital_DCG_objectives"] = d[6]
flush(stdout)

existing_plans = [[cg_data.suppression_plans.crews_present[i, j, :] 
                     for j=1:cg_data.suppression_plans.plans_per_fire[i]
                  ]
                  for i=1:NUM_FIRES 
                 ];

existing_routes = [[cg_data.routes.fires_fought[i, j, :, :] 
                     for j=1:cg_data.routes.routes_per_crew[i]
                  ]
                  for i=1:NUM_CREWS
                 ]

for excess in [-2, -1, 0, 1, 2]

    pattern = [(excess, 1e30)]
    params["int_aware_price_adjustment"] = [pattern for i in 1:params["max_iters"]]

    d2, cg_data2 = run_tests(params)
    println("Iterations: $(d2[4])")
    flush(stdout)

    for fire in 1:NUM_FIRES

        for plan_ix in 1:cg_data2.suppression_plans.plans_per_fire[fire] 

            if value(d2[9]["plan"][fire, plan_ix]) > fire_thresh

                plan = cg_data2.suppression_plans.crews_present[fire, plan_ix, :]
                cost = cg_data2.suppression_plans.plan_costs[fire, plan_ix]

                if ~(plan in existing_plans[fire])

                    push!(existing_plans[fire], plan)

                    update_available_supp_plans(fire, cost, plan, cg_data.suppression_plans)
                    push!(p_per_f, ((fire, cg_data.suppression_plans.plans_per_fire[fire]), 
                                       value(d2[9]["plan"][fire, plan_ix])))
                end
            end
        end
    end 


    for crew in 1:NUM_CREWS

        for route_ix in 1:cg_data2.routes.routes_per_crew[crew] 

            if value(d2[9]["route"][crew, route_ix]) > crew_thresh

                route = cg_data2.routes.fires_fought[crew, route_ix, :, :]
                cost = cg_data2.routes.route_costs[crew, route_ix]
                oo_region = cg_data2.routes.out_of_reg[crew, route_ix, :]

                if ~(route in existing_routes[crew])

                    push!(existing_routes[crew], route)

                    update_available_crew_routes(crew, cost, route, oo_region, cg_data.routes)
                    push!(r_per_c, ((crew, cg_data.routes.routes_per_crew[crew]), 
                          value(d2[9]["route"][crew, route_ix])))
                end

            end
        end
    end 
end

config = Dict{String,Any}("ws_dual" => false)
config["master_problem"] = Dict{String,Any}()

config["master_problem"]["dual_stabilization"] = false
config["master_problem"]["dual_secondary_eps"] = false
config["master_problem"]["dual_warm_start"] = false

r_data = RegionData(convert.(Int, ones(NUM_CREWS)), convert.(Int, ones(100)))
rotation_order = get_rotation_orders(r_data.crew_regions)

form_time = @elapsed pb = master_problem(config, cg_data.routes, cg_data.suppression_plans, 
                                         r_data, rotation_order, 0, true)
set_optimizer_attribute(pb["m"], "TimeLimit", 300)
sol_time = @elapsed optimize!(pb["m"])

pb_allot = convert.(Int, round.(100 .* get_fire_allotments(pb, cg_data)) ./ 100)

println()
println("Restore integrality")
println("___________________")
num_vars = num_variables(pb["m"])
println("Number of variables: $num_vars")
println("Formulate time: $form_time")
println("Solve time: $sol_time")
val = objective_value(pb["m"])
println("Objective value: $val")
println("Gap : $(val/d[6]["master_problem"] - 1)")
println()
println("*********************************************")
flush(stdout)

here
Fires : 3
Crews : 10

Running first DCG
__________
Iterations: 48
Objective values: Dict("price_and_branch" => 1.409384091836006e6, "master_problem" => 1.138800534771232e6)
Timings: Dict("init_cg" => 0.1344548, "price_and_branch" => 0.1295728, "cg" => 1.533599100000001, "ws" => 0.5434583)

Running integraity-aware DCG
__________
Iterations: 23
Iterations: 21
Iterations: 17
Iterations: 36
Iterations: 37

Restore integrality
___________________
Number of variables: 1346
Formulate time: 0.4596211
Solve time: 0.5713301
Objective value: 1.1672659340711462e6
Gap : 0.024995948307692473

*********************************************


In [152]:
function run_tests(test_features)
    
    in_path = test_features["in_path"]

    # get inital fire perimeters and no-suppression progression parameters
    M = readdlm(in_path * "/sample_growth_patterns.csv", ',')
    start_perims = M[:, 1]
    progressions = M[:, 2:15]

    global NUM_TIME_PERIODS = size(M)[2] - 1 
    global NUM_FIRES = test_features["num_fires"]
    global NUM_CREWS = test_features["num_crews"]
    global LINE_PER_CREW = test_features["line_per_crew"]

    g_data, crew_status = load_data(in_path)
    r_data = RegionData(convert.(Int, ones(NUM_CREWS)), convert.(Int, ones(100)))

    rotation_order = get_rotation_orders(r_data.crew_regions)
    A = generate_arcs(g_data, r_data, crew_status);

    rest_pen = get_rest_penalties(crew_status.rest_by, 1e10, positive)
    cost_params = Dict("cost_per_mile"=> 1, "rest_violation" => rest_pen, "fight_fire" => ALPHA)
    arc_costs = get_arc_costs(g_data, A, cost_params)

    c_data = define_network_constraint_data(A)
    
    fire_configs = []
    for fire in 1:NUM_FIRES
        model_config = Dict("model_type" => "simple_linear", "progressions" => progressions[fire, :], 
                            "start_perim" => start_perims[fire], "line_per_crew" => LINE_PER_CREW, 
                            "beta" => BETA)
        push!(fire_configs, model_config)
    end 
    
    gamma = test_features["gamma"]
    
    v_s = [0]
    e_s = [0]
    
    rhos = []
    allotment_history = []
    
    global AGG_PREC = test_features["agg_prec"]
    global PASSIVE_STATES =  test_features["passive_states"]
    
    ts = Dict{String, Float64}()
    best_sols = Dict{String, Float64}()
    allotments = Dict{String, Any}()
    
    fire_solver_configs = [Dict{String,Any}("solver_type" => test_features["fire_solver_type"]) for fire in 1:NUM_FIRES]
    
    max_plans = 1000
    ts["init_cg"] = @elapsed col_gen_data = initialize_column_generation(A, arc_costs, c_data, fire_configs, 
                                                                        fire_solver_configs, max_plans)
    
    
    if test_features["fire_solver_type"] == "dp"
        v_s = [size(col_gen_data.plan_sps[i]["graphs"]["ceiling"])[1] for i in 1:NUM_FIRES]

        e_s = [sum([length(col_gen_data.plan_sps[k]["graphs"]["ceiling"][i, j]) 
                  for i in 1:size(col_gen_data.plan_sps[k]["graphs"]["ceiling"])[1],
                      j in 1:size(col_gen_data.plan_sps[k]["graphs"]["ceiling"])[2]
                      if isassigned(col_gen_data.plan_sps[k]["graphs"]["ceiling"], i, j)
                    ]
                   )
               for k = 1:NUM_FIRES]
    end
    
    global AGG_PREC = 30
    global PASSIVE_STATES = 5
    
    if (test_features["apply_warm_start"]) | (test_features["dual_warm_start"] == "calculate")
        
        println("here")
        ts["ws"] = @elapsed ws, ws_dict = warm_start_suppression_plans(10, fire_configs, r_data, c_data, rotation_order, arc_costs, 
                                                             gamma, false, false, 3600)

        if test_features["dual_warm_start"] == "calculate"
            test_features["dual_warm_start"] = dual.(ws_dict["f_linking"])
        end

        if test_features["apply_warm_start"]
            for fire in 1:NUM_FIRES
                ws_fire = ws[fire]
                for plan in ws_fire
                    cost = get_fire_cost(plan, fire_configs[fire])
                    update_available_supp_plans(fire, cost, plan, col_gen_data.suppression_plans)
                end
            end
        end
    end

    
    global AGG_PREC = test_features["agg_prec"]
    global PASSIVE_STATES =  test_features["passive_states"]



    objs = []
    max_iters = test_features["max_iters"]
    n_iters = 0
    opt = false
    ts["cg"] = 0
    max_time = test_features["cg_time_limit"]
    col_gen_config = Dict{String,Any}("ws_dual" => false)
    col_gen_config["master_problem"] = Dict{String,Any}()
    
    col_gen_config["master_problem"]["dual_stabilization"] = test_features["dual_stab_type"]
    col_gen_config["master_problem"]["dual_secondary_eps"] = test_features["dual_stab_eps"]
    col_gen_config["master_problem"]["dual_warm_start"] = test_features["dual_warm_start"]
        
    if test_features["dual_stab_anchor"] == "all_ones"
        col_gen_config["master_problem"]["dual_warm_start"] = ones(size(col_gen_config["master_problem"]["dual_warm_start"]))
    end
    mp = master_problem(col_gen_config, col_gen_data.routes, col_gen_data.suppression_plans, 
                    r_data, rotation_order, 0, false)
    
    current_num_routes = copy(col_gen_data.routes.routes_per_crew)
    current_num_plans = copy(col_gen_data.suppression_plans.plans_per_fire)
        

    
    while (~opt) & (n_iters < max_iters) & (ts["cg"] < max_time)

        n_iters += 1
        dual_stab_weight = test_features["ws_dual_weight"][n_iters] + test_features["int_aware_dual_weight"][n_iters]
        col_gen_config["ws_dual_weight"] = dual_stab_weight
        col_gen_config["ws_dual"] = test_features["ws_dual_weight"][n_iters] * col_gen_config["master_problem"]["dual_warm_start"]
        if dual_stab_weight > 0
            col_gen_config["ws_dual"] = col_gen_config["ws_dual"] / dual_stab_weight
        end
        
        for fire in 1:NUM_FIRES
            fire_solver_configs[fire]["int_aware_adjustment_pattern"] = test_features["int_aware_price_adjustment"][n_iters]
            if "int_aware_capacities" in keys(test_features)
                fire_solver_configs[fire]["int_aware_capacities"] = test_features["int_aware_capacities"]
            end
        end

        ts["cg"] += @elapsed mp, a, b, c = run_CG_step(col_gen_data, A, arc_costs, g_data, r_data, fire_configs, 
                                              fire_solver_configs, col_gen_config, rotation_order, gamma,  
                                              test_features["restore_cost"], mp)

        push!(objs, a)
        push!(rhos, b)
        push!(allotment_history, c)

        next_num_routes = col_gen_data.routes.routes_per_crew
        next_num_plans = col_gen_data.suppression_plans.plans_per_fire

        if (sum(next_num_routes) == sum(current_num_routes)) & (sum(next_num_plans) == sum(current_num_plans))
            opt = true
        end

        current_num_routes = copy(next_num_routes)
        current_num_plans = copy(next_num_plans)

    end
    
    # no more stabilization for assessment
    col_gen_config["master_problem"]["dual_stabilization"] = false
    
    # get price and branch stats
    pb = master_problem(col_gen_config, col_gen_data.routes, col_gen_data.suppression_plans, 
                        r_data, rotation_order, gamma, true)
    ts["price_and_branch"] = @elapsed optimize!(pb["m"])
    
    best_sols["price_and_branch"] = objective_value(pb["m"])
    
    plans_chosen = [i for i in eachindex(pb["plan"]) if value(pb["plan"][i]) > 0.5]
    
    pb_allotment = convert.(Int8, zeros(size(col_gen_data.suppression_plans.crews_present[:, 1, :])))
    for plan in plans_chosen
        pb_allotment[plan[1], :] = col_gen_data.suppression_plans.crews_present[plan[1], plan[2], :]
    end
    
    allotments["price_and_branch"] = pb_allotment
    
    mp = master_problem(col_gen_config, col_gen_data.routes, col_gen_data.suppression_plans, 
                        r_data, rotation_order, gamma, false)
    optimize!(mp["m"])

    best_sols["master_problem"] = objective_value(mp["m"])
    allotments["master_problem"] = get_fire_allotments(mp, col_gen_data)
    
    if test_features["solve_explicit_int_time_limit"] > 0
        
        tl = test_features["solve_explicit_int_time_limit"]
        m, p, l, z, q, oor, linking = full_formulation(true, r_data, c_data, rotation_order, arc_costs, 
                                  progressions, start_perims, BETA, gamma, false, tl)
        ts["explicit_int"] = @elapsed optimize!(m)
        
        if has_values(m)
            best_sols["explicit_int"] = objective_value(m)
            allotments["explicit_int"] = convert.(Int8, ceil.(value.(l) / LINE_PER_CREW  .- 0.001))
        else
            best_sols["explicit_int"] = false
        end
            
    end

    
    if test_features["solve_explicit_lin_time_limit"] > 0
        
        tl = test_features["solve_explicit_lin_time_limit"]
    
        m2, p, l, z, q, oor, linking = full_formulation(false, r_data, c_data, rotation_order, arc_costs, 
                                  progressions, start_perims, BETA, gamma, false, tl)
        ts["explicit_lr"] = @elapsed optimize!(m2)
        
        if has_values(m2)
            best_sols["explicit_lr"] = objective_value(m2)
            allotments["explicit_lr"] = value.(l) / LINE_PER_CREW
        else
            best_sols["explicit_lr"] = false
        end
    
    end
    
    if test_features["solve_net_flow_time_limit"] > 0
        tl = test_features["solve_net_flow_time_limit"]
        ts["net_flow_int"] = @elapsed ws, ws_dict = warm_start_suppression_plans(10, fire_configs, r_data, c_data, rotation_order, arc_costs, 
                                                     gamma, true, false, tl)
        if has_values(ws_dict["m"])
            best_sols["net_flow_int"] = objective_value(ws_dict["m"])
            best_sols["net_flow_int_bound"] = objective_bound(ws_dict["m"])
            allotments["net_flow_int"] = value.(ws_dict["d"])
            allotments["net_flow_int_full_data"] = ws_dict
        else
            best_sols["net_flow_int"] = false
        end
    end
            
    
    return [fire_solver_configs[1]["solver_type"], sum(v_s), sum(e_s), n_iters, ts, best_sols, allotments, rhos, mp, allotment_history], col_gen_data

end

run_tests (generic function with 1 method)

In [6]:
fc = (6, 20)
    
fire_thresh = -0.0001
crew_thresh = -0.0001

params = Dict{String, Any}()
params["gamma"] = 0

params["in_path"] = "data/raw/big_fire"
params["agg_prec"] = 10
params["passive_states"] = 30
params["num_fires"] = fc[1]
params["num_crews"] = fc[2]
params["fire_solver_type"] = "dp"

params["apply_warm_start"] = false
params["restore_cost"] = false

params["solve_explicit_lin_time_limit"] = 0
params["solve_explicit_int_time_limit"] = 0
params["cg_time_limit"] = 300
params["solve_net_flow_time_limit"] = 0

params["max_iters"] = 1000

params["dual_stab_type"] = "global"
params["dual_stab_anchor"] = false
params["dual_stab_eps"] = 0.01

params["ws_dual_weight"] = zeros(params["max_iters"])
params["int_aware_dual_weight"] = zeros(params["max_iters"])

pattern = [(0, 0)]
params["int_aware_price_adjustment"] = [pattern for i in 1:params["max_iters"]]


r_per_c = []
p_per_f = []

d, cg_data = run_tests(params)


params["int_aware_capacities"] = d[7]["master_problem"]

println("Fires : $(params["num_fires"])")
println("Crews : $(params["num_crews"])")
println()

println("Running first DCG")
println("__________")
println("Iterations: $(d[4])")
println("Objective values: $(d[6])")
println("Timings: $(d[5])")
println()

println("Running integraity-aware DCG")
println("__________")
flush(stdout)

existing_plans = [[cg_data.suppression_plans.crews_present[i, j, :] 
                     for j=1:cg_data.suppression_plans.plans_per_fire[i]
                  ]
                  for i=1:NUM_FIRES 
                 ];

existing_routes = [[cg_data.routes.fires_fought[i, j, :, :] 
                     for j=1:cg_data.routes.routes_per_crew[i]
                  ]
                  for i=1:NUM_CREWS
                 ]

for excess in [-2, -1, 0, 1, 2]

    pattern = [(excess, 1e30)]
    params["int_aware_price_adjustment"] = [pattern for i in 1:params["max_iters"]]

    d2, cg_data2 = run_tests(params)
    println("Iterations: $(d2[4])")
    flush(stdout)

    for fire in 1:NUM_FIRES

        for plan_ix in 1:cg_data2.suppression_plans.plans_per_fire[fire] 

            if value(d2[9]["plan"][fire, plan_ix]) > fire_thresh

                plan = cg_data2.suppression_plans.crews_present[fire, plan_ix, :]
                cost = cg_data2.suppression_plans.plan_costs[fire, plan_ix]

                if ~(plan in existing_plans[fire])

                    push!(existing_plans[fire], plan)

                    update_available_supp_plans(fire, cost, plan, cg_data.suppression_plans)
                    push!(p_per_f, ((fire, cg_data.suppression_plans.plans_per_fire[fire]), 
                                       value(d2[9]["plan"][fire, plan_ix])))
                end
            end
        end
    end 


    for crew in 1:NUM_CREWS

        for route_ix in 1:cg_data2.routes.routes_per_crew[crew] 

            if value(d2[9]["route"][crew, route_ix]) > crew_thresh

                route = cg_data2.routes.fires_fought[crew, route_ix, :, :]
                cost = cg_data2.routes.route_costs[crew, route_ix]
                oo_region = cg_data2.routes.out_of_reg[crew, route_ix, :]

                if ~(route in existing_routes[crew])

                    push!(existing_routes[crew], route)

                    update_available_crew_routes(crew, cost, route, oo_region, cg_data.routes)
                    push!(r_per_c, ((crew, cg_data.routes.routes_per_crew[crew]), 
                          value(d2[9]["route"][crew, route_ix])))
                end

            end
        end
    end 
end

config = Dict{String,Any}("ws_dual" => false)
config["master_problem"] = Dict{String,Any}()

config["master_problem"]["dual_stabilization"] = false
config["master_problem"]["dual_secondary_eps"] = false
config["master_problem"]["dual_warm_start"] = false

r_data = RegionData(convert.(Int, ones(NUM_CREWS)), convert.(Int, ones(100)))
rotation_order = get_rotation_orders(r_data.crew_regions)

form_time = @elapsed pb = master_problem(config, cg_data.routes, cg_data.suppression_plans, 
                                         r_data, rotation_order, 0, true)
set_optimizer_attribute(pb["m"], "TimeLimit", 300)
sol_time = @elapsed optimize!(pb["m"])

pb_allot = convert.(Int, round.(100 .* get_fire_allotments(pb, cg_data)) ./ 100)

println()
println("Restore integrality")
println("___________________")
num_vars = num_variables(pb["m"])
println("Number of variables: $num_vars")
println("Formulate time: $form_time")
println("Solve time: $sol_time")
val = objective_value(pb["m"])
println("Objective value: $val")
println("Gap : $(val/d[6]["master_problem"] - 1)")
println()
println("*********************************************")
flush(stdout)

Fires : 6
Crews : 20

Running first DCG
__________
Iterations: 46
Objective values: Dict("price_and_branch" => 3.633108982511832e6, "master_problem" => 2.671221211544192e6)
Timings: Dict("init_cg" => 0.5733929, "price_and_branch" => 0.0979713, "cg" => 7.095825499999999, "ws" => 2.3538059)

Running integraity-aware DCG
__________
Iterations: 30
Iterations: 32
Iterations: 34
Iterations: 61
Iterations: 52

Restore integrality
___________________
Number of variables: 4730
Formulate time: 3.6940849
Solve time: 6.9258758
Objective value: 2.761300847772844e6
Gap : 0.033722267492993785

*********************************************


In [178]:
fc = (9, 30)
    
fire_thresh = -0.0001
crew_thresh = -0.0001

params = Dict{String, Any}()
params["gamma"] = 0

params["in_path"] = "data/raw/big_fire"
params["agg_prec"] = 10
params["passive_states"] = 30
params["num_fires"] = fc[1]
params["num_crews"] = fc[2]
params["fire_solver_type"] = "dp"

params["apply_warm_start"] = false
params["restore_cost"] = false

params["solve_explicit_lin_time_limit"] = 0
params["solve_explicit_int_time_limit"] = 0
params["cg_time_limit"] = 300
params["solve_net_flow_time_limit"] = 0

params["max_iters"] = 1000

params["dual_stab_type"] = "global"
params["dual_stab_anchor"] = false
params["dual_stab_eps"] = 0.01

params["ws_dual_weight"] = zeros(params["max_iters"])
params["int_aware_dual_weight"] = zeros(params["max_iters"])

pattern = [(0, 0)]
params["int_aware_price_adjustment"] = [pattern for i in 1:params["max_iters"]]


r_per_c = []
p_per_f = []

d, cg_data = run_tests(params)


params["int_aware_capacities"] = d[7]["master_problem"]

println("Fires : $(params["num_fires"])")
println("Crews : $(params["num_crews"])")
println()

println("Running first DCG")
println("__________")
println("Iterations: $(d[4])")
println("Objective values: $(d[6])")
println("Timings: $(d[5])")
println()

println("Running integraity-aware DCG")
println("__________")
flush(stdout)

existing_plans = [[cg_data.suppression_plans.crews_present[i, j, :] 
                     for j=1:cg_data.suppression_plans.plans_per_fire[i]
                  ]
                  for i=1:NUM_FIRES 
                 ];

existing_routes = [[cg_data.routes.fires_fought[i, j, :, :] 
                     for j=1:cg_data.routes.routes_per_crew[i]
                  ]
                  for i=1:NUM_CREWS
                 ]

for excess in [-2, -1, 0, 1, 2]

    pattern = [(excess, 1e30)]
    params["int_aware_price_adjustment"] = [pattern for i in 1:params["max_iters"]]

    d2, cg_data2 = run_tests(params)
    println("Iterations: $(d2[4])")
    flush(stdout)

    for fire in 1:NUM_FIRES

        for plan_ix in 1:cg_data2.suppression_plans.plans_per_fire[fire] 

            if value(d2[9]["plan"][fire, plan_ix]) > fire_thresh

                plan = cg_data2.suppression_plans.crews_present[fire, plan_ix, :]
                cost = cg_data2.suppression_plans.plan_costs[fire, plan_ix]

                if ~(plan in existing_plans[fire])

                    push!(existing_plans[fire], plan)

                    update_available_supp_plans(fire, cost, plan, cg_data.suppression_plans)
                    push!(p_per_f, ((fire, cg_data.suppression_plans.plans_per_fire[fire]), 
                                       value(d2[9]["plan"][fire, plan_ix])))
                end
            end
        end
    end 


    for crew in 1:NUM_CREWS

        for route_ix in 1:cg_data2.routes.routes_per_crew[crew] 

            if value(d2[9]["route"][crew, route_ix]) > crew_thresh

                route = cg_data2.routes.fires_fought[crew, route_ix, :, :]
                cost = cg_data2.routes.route_costs[crew, route_ix]
                oo_region = cg_data2.routes.out_of_reg[crew, route_ix, :]

                if ~(route in existing_routes[crew])

                    push!(existing_routes[crew], route)

                    update_available_crew_routes(crew, cost, route, oo_region, cg_data.routes)
                    push!(r_per_c, ((crew, cg_data.routes.routes_per_crew[crew]), 
                          value(d2[9]["route"][crew, route_ix])))
                end

            end
        end
    end 
end

config = Dict{String,Any}("ws_dual" => false)
config["master_problem"] = Dict{String,Any}()

config["master_problem"]["dual_stabilization"] = false
config["master_problem"]["dual_secondary_eps"] = false
config["master_problem"]["dual_warm_start"] = false

r_data = RegionData(convert.(Int, ones(NUM_CREWS)), convert.(Int, ones(100)))
rotation_order = get_rotation_orders(r_data.crew_regions)

form_time = @elapsed pb = master_problem(config, cg_data.routes, cg_data.suppression_plans, 
                                         r_data, rotation_order, 0, true)
set_optimizer_attribute(pb["m"], "TimeLimit", 300)
sol_time = @elapsed optimize!(pb["m"])

pb_allot = convert.(Int, round.(100 .* get_fire_allotments(pb, cg_data)) ./ 100)

println()
println("Restore integrality")
println("___________________")
num_vars = num_variables(pb["m"])
println("Number of variables: $num_vars")
println("Formulate time: $form_time")
println("Solve time: $sol_time")
val = objective_value(pb["m"])
println("Objective value: $val")
println("Gap : $(val/d[6]["master_problem"] - 1)")
println()
println("*********************************************")
flush(stdout)

Fires : 9
Crews : 30

Running first DCG
__________
Iterations: 49
Objective values: Dict("price_and_branch" => 9.018218008705923e9, "master_problem" => 4.3067503718116e6)
Timings: Dict("init_cg" => 1.5190441, "price_and_branch" => 1.0377923, "cg" => 19.0913754, "ws" => 4.1544377)

Running integraity-aware DCG
__________
Iterations: 53
Iterations: 58
Iterations: 67
Iterations: 82
Iterations: 69

Restore integrality
___________________
Number of variables: 10646
Formulate time: 13.3446283
Solve time: 280.783688
Objective value: 4.40189063268633e6
Gap : 0.022090962480072696

*********************************************


In [179]:
routes_used = [i for i in eachindex(pb["route"]) if value(pb["route"][i]) > 0.9]
routes_generated = [i for i in r_per_c if i[1] in routes_used]
Dict("original_dcg" => NUM_CREWS - length(routes_generated), 
     "later_dcg_unused" => length([i for i in routes_generated if i[2] < 0.001]),
     "later_dcg_used" => length([i for i in routes_generated if i[2] >= 0.001]))

Dict{String, Int64} with 3 entries:
  "later_dcg_used"   => 2
  "later_dcg_unused" => 20
  "original_dcg"     => 8

In [180]:
plans_used = [i for i in eachindex(pb["plan"]) if value(pb["plan"][i]) > 0.9]
plans_generated = [i for i in p_per_f if i[1] in plans_used]
Dict("original_dcg" => NUM_FIRES - length(plans_generated), 
     "later_dcg_unused" => length([i for i in plans_generated if i[2] < 0.001]),
     "later_dcg_used" => length([i for i in plans_generated if i[2] >= 0.001]))

Dict{String, Int64} with 3 entries:
  "later_dcg_used"   => 2
  "later_dcg_unused" => 5
  "original_dcg"     => 2

In [181]:
for fire in 1:NUM_FIRES
    splans = cg_data.suppression_plans
    num_plans = splans.plans_per_fire[fire]
    adj_costs = splans.plan_costs[fire, 1:num_plans] .+ 
                splans.crews_present[fire, 1:num_plans, :] * dual.(d[9]["rho"])[fire, :] .-  dual.(d[9]["pi"])[fire, :]
    println(mean(adj_costs .> val - d[6]["master_problem"]))
end

0.061855670103092786
0.13385826771653545
0.625
0.43548387096774194
0.13297872340425532
0.031746031746031744
0.1791044776119403
0.4673913043478261
0.2222222222222222


In [182]:
for crew in 1:NUM_CREWS
    croutes = cg_data.routes
    num_routes = croutes.routes_per_crew[crew]
    adj_costs = [sum(croutes.fires_fought[crew, i, :, :]  .* dual.(d[9]["rho"])) for i in 1:num_routes]
    adj_costs = croutes.route_costs[crew, 1:num_routes] .- adj_costs .-  dual.(d[9]["sigma"])[crew, :]
    println(mean(adj_costs .> val - d[6]["master_problem"]))
end

0.07947019867549669
0.13915857605177995
0.1437308868501529
0.13975155279503104
0.10097719869706841
0.23510971786833856
0.11510791366906475
0.1245674740484429
0.12666666666666668
0.1258741258741259
0.10033444816053512
0.1794871794871795
0.1519756838905775
0.1437308868501529
0.15178571428571427
0.11935483870967742
0.07835820895522388
0.1391941391941392
0.10508474576271186
0.1347517730496454
0.15457413249211358
0.07419354838709677
0.14423076923076922
0.10060975609756098
0.15555555555555556
0.12152777777777778
0.06551724137931035
0.12
0.10714285714285714
0.10676156583629894


In [151]:
adj_costs[22]

1239.253248035122

In [152]:
adj_costs

196-element Vector{Float64}:
 241902.5773739722
    595.1808524034277
   2507.3682406253065
  24000.19233253345
  67046.88133109518
   3404.2286069145775
  23964.951536429493
   5257.9885886562115
  49708.636036850774
  10742.417513684311
  13786.772103347088
   9482.30860202969
   4802.467782525055
      ⋮
   1427.1775779320742
   8927.517161652358
   5298.937192458776
    627.4172335651529
    601.4681697984051
    542.894138388423
    544.1906126893882
    156.3746872798074
      3.536231066856999
      0.0
   8635.57134193179
    151.2816001563624

In [143]:
[i for i in eachindex(pb["route"]) if value(pb["route"][i]) > 0.9]

20-element Vector{Tuple{Int64, Int64}}:
 (16, 112)
 (2, 22)
 (12, 108)
 (18, 40)
 (13, 95)
 (7, 80)
 (6, 81)
 (9, 39)
 (1, 36)
 (10, 88)
 (20, 51)
 (17, 134)
 (8, 68)
 (5, 64)
 (11, 18)
 (14, 99)
 (15, 90)
 (4, 116)
 (3, 22)
 (19, 36)

In [69]:
cg_data.suppression_plans.plan_costs[fire, 1:10]

10-element Vector{Float64}:
      8.510703824215365e9
   5000.0
      1.0189378215259777e8
      1.0179738790686765e8
 100248.5748380035
  23947.36842105263
   7105.263157894737
  17631.57894736842
  26052.631578947367
  36578.94736842105

In [63]:
dual.(d[9]["pi"])

6-element Vector{Float64}:
      1.1575909547738698e6
 563610.8789418872
      1.1536147257428574e6
      2.150161696750889e6
      1.1173799088774088e6
      1.0289543824293832e6

In [67]:
dual.(d[9]["rho"])[fire, :]

14-element Vector{Float64}:
 115163.7030038062
  55018.06777535345
  18217.75342593089
      0.0
      0.0
      0.0
      0.0
      0.0
      0.0
      0.0
      0.0
      0.0
      0.0
      0.0

## Full procedure

In [56]:
for fc in [(6, 20)]
    
    fire_thresh = -0.0001
    crew_thresh = 1.0001

    params = Dict{String, Any}()
    params["gamma"] = 0

    params["in_path"] = "data/raw/big_fire"
    params["agg_prec"] = 10
    params["passive_states"] = 30
    params["num_fires"] = fc[1]
    params["num_crews"] = fc[2]
    params["fire_solver_type"] = "dp"

    params["apply_warm_start"] = false
    params["restore_cost"] = false

    params["solve_explicit_lin_time_limit"] = 0
    params["solve_explicit_int_time_limit"] = 0
    params["cg_time_limit"] = 300
    params["solve_net_flow_time_limit"] = 0

    params["max_iters"] = 1000

    params["dual_stab_type"] = "global"
    params["dual_stab_anchor"] = false
    params["dual_stab_eps"] = 0.01

    params["ws_dual_weight"] = zeros(params["max_iters"])
    params["int_aware_dual_weight"] = zeros(params["max_iters"])

    pattern = [(0, 0)]
    params["int_aware_price_adjustment"] = [pattern for i in 1:params["max_iters"]]

    d, cg_data = run_tests(params)
    params["int_aware_capacities"] = d[7]["master_problem"]

    println("Fires : $(params["num_fires"])")
    println("Crews : $(params["num_crews"])")
    println()

    println("Running first DCG")
    println("__________")
    println("Iterations: $(d[4])")
    println("Objective values: $(d[6])")
    println("Timings: $(d[5])")
    println()

    println("Running integraity-aware DCG")
    println("__________")
    flush(stdout)

    existing_plans = [[cg_data.suppression_plans.crews_present[i, j, :] 
                         for j=1:cg_data.suppression_plans.plans_per_fire[i]
                      ]
                      for i=1:NUM_FIRES 
                     ];

    existing_routes = [[cg_data.routes.fires_fought[i, j, :, :] 
                         for j=1:cg_data.routes.routes_per_crew[i]
                      ]
                      for i=1:NUM_CREWS
                     ]

    for excess in [-2, -1, 0, 1, 2]

        pattern = [(excess, 1e30)]
        params["int_aware_price_adjustment"] = [pattern for i in 1:params["max_iters"]]

        d2, cg_data2 = run_tests(params)
        println("Iterations: $(d2[4])")
        flush(stdout)

        for fire in 1:NUM_FIRES

            for plan_ix in 1:cg_data2.suppression_plans.plans_per_fire[fire] 

                if value(d2[9]["plan"][fire, plan_ix]) > fire_thresh

                    plan = cg_data2.suppression_plans.crews_present[fire, plan_ix, :]
                    cost = cg_data2.suppression_plans.plan_costs[fire, plan_ix]

                    if ~(plan in existing_plans[fire])

                        push!(existing_plans[fire], plan)

                        update_available_supp_plans(fire, cost, plan, cg_data.suppression_plans)
                    end
                end
            end
        end 


        for crew in 1:NUM_CREWS

            for route_ix in 1:cg_data2.routes.routes_per_crew[crew] 

                if value(d2[9]["route"][crew, route_ix]) > crew_thresh

                    route = cg_data2.routes.fires_fought[crew, route_ix, :, :]
                    cost = cg_data2.routes.route_costs[crew, route_ix]
                    oo_region = cg_data2.routes.out_of_reg[crew, route_ix, :]

                    if ~(route in existing_routes[crew])

                        push!(existing_routes[crew], route)

                        update_available_crew_routes(crew, cost, route, oo_region, cg_data.routes)
                    end

                end
            end
        end 
    end

    config = Dict{String,Any}("ws_dual" => false)
    config["master_problem"] = Dict{String,Any}()

    config["master_problem"]["dual_stabilization"] = false
    config["master_problem"]["dual_secondary_eps"] = false
    config["master_problem"]["dual_warm_start"] = false

    r_data = RegionData(convert.(Int, ones(NUM_CREWS)), convert.(Int, ones(100)))
    rotation_order = get_rotation_orders(r_data.crew_regions)

    form_time = @elapsed pb = master_problem(config, cg_data.routes, cg_data.suppression_plans, 
                                             r_data, rotation_order, 0, true)
    set_optimizer_attribute(pb["m"], "TimeLimit", 300)
    sol_time = @elapsed optimize!(pb["m"])

    pb_allot = convert.(Int, round.(100 .* get_fire_allotments(pb, cg_data)) ./ 100)

    println()
    println("Restore integrality")
    println("___________________")
    num_vars = num_variables(pb["m"])
    println("Number of variables: $num_vars")
    println("Formulate time: $form_time")
    println("Solve time: $sol_time")
    val = objective_value(pb["m"])
    println("Objective value: $val")
    println("Gap : $(val/d[6]["master_problem"] - 1)")
    println()
    println("*********************************************")
    flush(stdout)
end

Fires : 6
Crews : 20

Running first DCG
__________
Iterations: 46
Objective values: Dict("price_and_branch" => 3.633108982511832e6, "master_problem" => 2.671221211544192e6)
Timings: Dict("init_cg" => 0.5608259, "price_and_branch" => 0.2708204, "cg" => 9.7143244, "ws" => 1.8870813)

Running integraity-aware DCG
__________
Iterations: 30
Iterations: 32
Iterations: 34
Iterations: 61
Iterations: 52

Restore integrality
___________________
Number of variables: 1669
Formulate time: 0.7488398
Solve time: 1.5407393
Objective value: 2.792618766942195e6
Gap : 0.04544646279138553

*********************************************


In [235]:
for fc in [(3, 10), (6, 20), (9, 30), (12, 40)]
    
    fire_thresh = -0.0001
    crew_thresh = 0.0001

    params = Dict{String, Any}()
    params["gamma"] = 0

    params["in_path"] = "data/raw/big_fire"
    params["agg_prec"] = 10
    params["passive_states"] = 30
    params["num_fires"] = fc[1]
    params["num_crews"] = fc[2]
    params["fire_solver_type"] = "dp"

    params["apply_warm_start"] = false
    params["restore_cost"] = false

    params["solve_explicit_lin_time_limit"] = 0
    params["solve_explicit_int_time_limit"] = 0
    params["cg_time_limit"] = 300
    params["solve_net_flow_time_limit"] = 0

    params["max_iters"] = 1000

    params["dual_stab_type"] = "global"
    params["dual_stab_anchor"] = false
    params["dual_stab_eps"] = 0.01

    params["ws_dual_weight"] = zeros(params["max_iters"])
    params["int_aware_dual_weight"] = zeros(params["max_iters"])

    pattern = [(0, 0)]
    params["int_aware_price_adjustment"] = [pattern for i in 1:params["max_iters"]]

    d, cg_data = run_tests(params)
    params["int_aware_capacities"] = d[7]["master_problem"]

    println("Fires : $(params["num_fires"])")
    println("Crews : $(params["num_crews"])")
    println()

    println("Running first DCG")
    println("__________")
    println("Iterations: $(d[4])")
    println("Objective values: $(d[6])")
    println("Timings: $(d[5])")
    println()

    println("Running integraity-aware DCG")
    println("__________")
    flush(stdout)

    existing_plans = [[cg_data.suppression_plans.crews_present[i, j, :] 
                         for j=1:cg_data.suppression_plans.plans_per_fire[i]
                      ]
                      for i=1:NUM_FIRES 
                     ];

    existing_routes = [[cg_data.routes.fires_fought[i, j, :, :] 
                         for j=1:cg_data.routes.routes_per_crew[i]
                      ]
                      for i=1:NUM_CREWS
                     ]

    for excess in [-2, -1, 0, 1, 2]

        pattern = [(excess, 1e30)]
        params["int_aware_price_adjustment"] = [pattern for i in 1:params["max_iters"]]

        d2, cg_data2 = run_tests(params)
        println("Iterations: $(d2[4])")
        flush(stdout)

        for fire in 1:NUM_FIRES

            for plan_ix in 1:cg_data2.suppression_plans.plans_per_fire[fire] 

                if value(d2[9]["plan"][fire, plan_ix]) > fire_thresh

                    plan = cg_data2.suppression_plans.crews_present[fire, plan_ix, :]
                    cost = cg_data2.suppression_plans.plan_costs[fire, plan_ix]

                    if ~(plan in existing_plans[fire])

                        push!(existing_plans[fire], plan)

                        update_available_supp_plans(fire, cost, plan, cg_data.suppression_plans)
                    end
                end
            end
        end 


        for crew in 1:NUM_CREWS

            for route_ix in 1:cg_data2.routes.routes_per_crew[crew] 

                if value(d2[9]["route"][crew, route_ix]) > crew_thresh

                    route = cg_data2.routes.fires_fought[crew, route_ix, :, :]
                    cost = cg_data2.routes.route_costs[crew, route_ix]
                    oo_region = cg_data2.routes.out_of_reg[crew, route_ix, :]

                    if ~(route in existing_routes[crew])

                        push!(existing_routes[crew], route)

                        update_available_crew_routes(crew, cost, route, oo_region, cg_data.routes)
                    end

                end
            end
        end 
    end

    config = Dict{String,Any}("ws_dual" => false)
    config["master_problem"] = Dict{String,Any}()

    config["master_problem"]["dual_stabilization"] = false
    config["master_problem"]["dual_secondary_eps"] = false
    config["master_problem"]["dual_warm_start"] = false

    r_data = RegionData(convert.(Int, ones(NUM_CREWS)), convert.(Int, ones(100)))
    rotation_order = get_rotation_orders(r_data.crew_regions)

    form_time = @elapsed pb = master_problem(config, cg_data.routes, cg_data.suppression_plans, 
                                             r_data, rotation_order, 0, true)
    set_optimizer_attribute(pb["m"], "TimeLimit", 300)
    sol_time = @elapsed optimize!(pb["m"])

    pb_allot = convert.(Int, round.(100 .* get_fire_allotments(pb, cg_data)) ./ 100)

    println()
    println("Restore integrality")
    println("___________________")
    num_vars = num_variables(pb["m"])
    println("Number of variables: $num_vars")
    println("Formulate time: $form_time")
    println("Solve time: $sol_time")
    val = objective_value(pb["m"])
    println("Objective value: $val")
    println("Gap : $(val/d[6]["master_problem"] - 1)")
    println()
    println("*********************************************")
    flush(stdout)
end

Fires : 3
Crews : 10

Running first DCG
__________
Iterations: 48
Objective values: Dict("price_and_branch" => 1.409384091836006e6, "master_problem" => 1.138800534771232e6)
Timings: Dict("init_cg" => 0.1235671, "price_and_branch" => 0.0330664, "cg" => 1.4205303999999996, "ws" => 0.5435866)

Running integraity-aware DCG
__________
Iterations: 23
Iterations: 21
Iterations: 17
Iterations: 36
Iterations: 37

Restore integrality
___________________
Number of variables: 690
Formulate time: 0.133513
Solve time: 0.2263785
Objective value: 1.1873281507401261e6
Gap : 0.04261291989877991

*********************************************
Fires : 6
Crews : 20

Running first DCG
__________
Iterations: 46
Objective values: Dict("price_and_branch" => 3.633108982511832e6, "master_problem" => 2.671221211544192e6)
Timings: Dict("init_cg" => 0.5337115, "price_and_branch" => 0.099827, "cg" => 6.560796400000001, "ws" => 1.9022789)

Running integraity-aware DCG
__________
Iterations: 30
Iterations: 32
Iteration

In [237]:
for fc in [(3, 10), (6, 20), (9, 30), (12, 40)]
    
    fire_thresh = -0.0001
    crew_thresh = -0.0001

    params = Dict{String, Any}()
    params["gamma"] = 0

    params["in_path"] = "data/raw/big_fire"
    params["agg_prec"] = 10
    params["passive_states"] = 30
    params["num_fires"] = fc[1]
    params["num_crews"] = fc[2]
    params["fire_solver_type"] = "dp"

    params["apply_warm_start"] = false
    params["restore_cost"] = false

    params["solve_explicit_lin_time_limit"] = 0
    params["solve_explicit_int_time_limit"] = 0
    params["cg_time_limit"] = 300
    params["solve_net_flow_time_limit"] = 0

    params["max_iters"] = 1000

    params["dual_stab_type"] = "global"
    params["dual_stab_anchor"] = false
    params["dual_stab_eps"] = 0.01

    params["ws_dual_weight"] = zeros(params["max_iters"])
    params["int_aware_dual_weight"] = zeros(params["max_iters"])

    pattern = [(0, 0)]
    params["int_aware_price_adjustment"] = [pattern for i in 1:params["max_iters"]]

    d, cg_data = run_tests(params)
    params["int_aware_capacities"] = d[7]["master_problem"]

    println("Fires : $(params["num_fires"])")
    println("Crews : $(params["num_crews"])")
    println()

    println("Running first DCG")
    println("__________")
    println("Iterations: $(d[4])")
    println("Objective values: $(d[6])")
    println("Timings: $(d[5])")
    println()

    println("Running integraity-aware DCG")
    println("__________")
    flush(stdout)

    existing_plans = [[cg_data.suppression_plans.crews_present[i, j, :] 
                         for j=1:cg_data.suppression_plans.plans_per_fire[i]
                      ]
                      for i=1:NUM_FIRES 
                     ];

    existing_routes = [[cg_data.routes.fires_fought[i, j, :, :] 
                         for j=1:cg_data.routes.routes_per_crew[i]
                      ]
                      for i=1:NUM_CREWS
                     ]

    for excess in [-2, -1, 0, 1, 2]

        pattern = [(excess, 1e30)]
        params["int_aware_price_adjustment"] = [pattern for i in 1:params["max_iters"]]

        d2, cg_data2 = run_tests(params)
        println("Iterations: $(d2[4])")
        flush(stdout)

        for fire in 1:NUM_FIRES

            for plan_ix in 1:cg_data2.suppression_plans.plans_per_fire[fire] 

                if value(d2[9]["plan"][fire, plan_ix]) > fire_thresh

                    plan = cg_data2.suppression_plans.crews_present[fire, plan_ix, :]
                    cost = cg_data2.suppression_plans.plan_costs[fire, plan_ix]

                    if ~(plan in existing_plans[fire])

                        push!(existing_plans[fire], plan)

                        update_available_supp_plans(fire, cost, plan, cg_data.suppression_plans)
                    end
                end
            end
        end 


        for crew in 1:NUM_CREWS

            for route_ix in 1:cg_data2.routes.routes_per_crew[crew] 

                if value(d2[9]["route"][crew, route_ix]) > crew_thresh

                    route = cg_data2.routes.fires_fought[crew, route_ix, :, :]
                    cost = cg_data2.routes.route_costs[crew, route_ix]
                    oo_region = cg_data2.routes.out_of_reg[crew, route_ix, :]

                    if ~(route in existing_routes[crew])

                        push!(existing_routes[crew], route)

                        update_available_crew_routes(crew, cost, route, oo_region, cg_data.routes)
                    end

                end
            end
        end 
    end

    config = Dict{String,Any}("ws_dual" => false)
    config["master_problem"] = Dict{String,Any}()

    config["master_problem"]["dual_stabilization"] = false
    config["master_problem"]["dual_secondary_eps"] = false
    config["master_problem"]["dual_warm_start"] = false

    r_data = RegionData(convert.(Int, ones(NUM_CREWS)), convert.(Int, ones(100)))
    rotation_order = get_rotation_orders(r_data.crew_regions)

    form_time = @elapsed pb = master_problem(config, cg_data.routes, cg_data.suppression_plans, 
                                             r_data, rotation_order, 0, true)
    set_optimizer_attribute(pb["m"], "TimeLimit", 300)
    sol_time = @elapsed optimize!(pb["m"])

    pb_allot = convert.(Int, round.(100 .* get_fire_allotments(pb, cg_data)) ./ 100)

    println()
    println("Restore integrality")
    println("___________________")
    num_vars = num_variables(pb["m"])
    println("Number of variables: $num_vars")
    println("Formulate time: $form_time")
    println("Solve time: $sol_time")
    val = objective_value(pb["m"])
    println("Objective value: $val")
    println("Gap : $(val/d[6]["master_problem"] - 1)")
    println()
    println("*********************************************")
    flush(stdout)
end

Fires : 3
Crews : 10

Running first DCG
__________
Iterations: 48
Objective values: Dict("price_and_branch" => 1.409384091836006e6, "master_problem" => 1.138800534771232e6)
Timings: Dict("init_cg" => 1.210148, "price_and_branch" => 0.115581, "cg" => 1.8509999, "ws" => 4.5911759)

Running integraity-aware DCG
__________
Iterations: 23
Iterations: 21
Iterations: 17
Iterations: 36
Iterations: 37

Restore integrality
___________________
Number of variables: 1346
Formulate time: 0.4650822
Solve time: 0.5986755
Objective value: 1.1672659340711462e6
Gap : 0.024995948307692473

*********************************************
Fires : 6
Crews : 20

Running first DCG
__________
Iterations: 46
Objective values: Dict("price_and_branch" => 3.633108982511832e6, "master_problem" => 2.671221211544192e6)
Timings: Dict("init_cg" => 0.9284345, "price_and_branch" => 0.1234502, "cg" => 6.9157966, "ws" => 2.1628266)

Running integraity-aware DCG
__________
Iterations: 30
Iterations: 32
Iterations: 34
Iteration

In [205]:
for excess in [-2, -1, 0, 1, 2]
    
    pattern = [(excess, 1e30)]
    params["int_aware_price_adjustment"] = [pattern for i in 1:params["max_iters"]]

    d2, cg_data2 = run_tests(params)
    println(d2[4])
    
    for fire in 1:NUM_FIRES
    
        for plan_ix in 1:cg_data2.suppression_plans.plans_per_fire[fire] 

            if value(d2[9]["plan"][fire, plan_ix]) > -0.0001

                plan = cg_data2.suppression_plans.crews_present[fire, plan_ix, :]
                cost = cg_data2.suppression_plans.plan_costs[fire, plan_ix]

                if ~(plan in existing_plans[fire])

                    push!(existing_plans[fire], plan)

                    update_available_supp_plans(fire, cost, plan, cg_data.suppression_plans)
                end
            end
        end
    end 


    for crew in 1:NUM_CREWS

        for route_ix in 1:cg_data2.routes.routes_per_crew[crew] 

            if value(d2[9]["route"][crew, route_ix]) > -0.0001

                route = cg_data2.routes.fires_fought[crew, route_ix, :, :]
                cost = cg_data2.routes.route_costs[crew, route_ix]
                oo_region = cg_data2.routes.out_of_reg[crew, route_ix, :]

                if ~(route in existing_routes[crew])

                    push!(existing_routes[crew], route)

                    update_available_crew_routes(crew, cost, route, oo_region, cg_data.routes)
                end

            end
        end
    end 
end

23
21
17
36
37


In [206]:
cg_data.suppression_plans.plans_per_fire

3-element Vector{Int64}:
 138
  87
  18

In [207]:
cg_data.routes.routes_per_crew

10-element Vector{Int64}:
 111
  92
  96
 100
 103
  90
  84
  92
  98
  87

In [208]:
config = Dict{String,Any}("ws_dual" => false)
config["master_problem"] = Dict{String,Any}()

config["master_problem"]["dual_stabilization"] = false
config["master_problem"]["dual_secondary_eps"] = false
config["master_problem"]["dual_warm_start"] = false

false

In [209]:
r_data = RegionData(convert.(Int, ones(NUM_CREWS)), convert.(Int, ones(100)))
rotation_order = get_rotation_orders(r_data.crew_regions)

form_time = @elapsed pb = master_problem(config, cg_data.routes, cg_data.suppression_plans, 
                                         r_data, rotation_order, 0, true)
set_optimizer_attribute(pb["m"], "TimeLimit", 300)
sol_time = @elapsed optimize!(pb["m"])

pb_allot = convert.(Int, round.(100 .* get_fire_allotments(pb, cg_data)) ./ 100)

form_time, sol_time, objective_value(pb["m"])

(0.5131061, 0.5078937, 1.1672659340711462e6)

In [170]:
pb["m"]

A JuMP Model
Minimization problem with:
Variables: 4730
Objective function type: AffExpr
`AffExpr`-in-`MathOptInterface.EqualTo{Float64}`: 20 constraints
`AffExpr`-in-`MathOptInterface.GreaterThan{Float64}`: 390 constraints
`VariableRef`-in-`MathOptInterface.GreaterThan{Float64}`: 4730 constraints
`VariableRef`-in-`MathOptInterface.Integer`: 4730 constraints
Model mode: AUTOMATIC
CachingOptimizer state: ATTACHED_OPTIMIZER
Solver name: Gurobi
Names registered in the model: linking, out_of_region, plan, plan_per_fire, q, route, route_per_crew

In [512]:
pb_allot

9×14 Matrix{Int64}:
 0   0   0  0  0  0  0  0  0   0   0  13  1  0
 0   0   0  6  3  3  6  6  6   0   0   0  0  0
 9   2   0  0  0  0  0  0  0   0   0   0  0  0
 9  13  17  2  0  0  0  0  0   0   0   0  0  0
 0   0   0  6  6  6  6  8  8   5   8   8  0  0
 3   0   0  0  1  0  0  0  0   0   2   1  0  0
 0   0   0  6  5  3  5  5  6   1   0   0  0  0
 9  15  13  2  0  0  0  0  0   0   0   0  0  0
 0   0   0  0  3  3  5  3  3  16  16   6  1  1

### Setup

In [115]:
in_path = params["in_path"]

# get inital fire perimeters and no-suppression progression parameters
M = readdlm(in_path * "/sample_growth_patterns.csv", ',')
start_perims = M[:, 1]
progressions = M[:, 2:15]

global NUM_TIME_PERIODS = size(M)[2] - 1 
global NUM_FIRES = params["num_fires"]
global NUM_CREWS = params["num_crews"]

g_data, crew_status = load_data(in_path)
r_data = RegionData(convert.(Int, ones(NUM_CREWS)), convert.(Int, ones(100)))

rotation_order = get_rotation_orders(r_data.crew_regions)
A = generate_arcs(g_data, r_data, crew_status);

rest_pen = get_rest_penalties(crew_status.rest_by, 1e10, positive)
cost_params = Dict("cost_per_mile"=> 1, "rest_violation" => rest_pen, "fight_fire" => ALPHA)
arc_costs = get_arc_costs(g_data, A, cost_params)

c_data = define_network_constraint_data(A)

fire_configs = []
for fire in 1:NUM_FIRES
    model_config = Dict("model_type" => "simple_linear", "progressions" => progressions[fire, :], 
                        "start_perim" => start_perims[fire], "line_per_crew" => LINE_PER_CREW, 
                        "beta" => BETA)
    push!(fire_configs, model_config)
end 

gamma = params["gamma"]


rhos = []
allotment_history = []

global AGG_PREC = params["agg_prec"]
global PASSIVE_STATES =  params["passive_states"]

ts = Dict{String, Float64}()
best_sols = Dict{String, Float64}()
allotments = Dict{String, Any}()

fire_solver_configs = [Dict{String,Any}("solver_type" => params["fire_solver_type"]) for fire in 1:NUM_FIRES]

max_plans = 1000
ts["init_cg"] = @elapsed col_gen_data = initialize_column_generation(A, arc_costs, c_data, fire_configs, 
                                                                    fire_solver_configs, max_plans)

if params["fire_solver_type"] == "dp"
    v_s = [size(col_gen_data.plan_sps[i]["graphs"]["ceiling"])[1] for i in 1:NUM_FIRES]

    e_s = [sum([length(col_gen_data.plan_sps[k]["graphs"]["ceiling"][i, j]) 
              for i in 1:size(col_gen_data.plan_sps[k]["graphs"]["ceiling"])[1],
                  j in 1:size(col_gen_data.plan_sps[k]["graphs"]["ceiling"])[2]
                  if isassigned(col_gen_data.plan_sps[k]["graphs"]["ceiling"], i, j)
                ]
               )
           for k = 1:NUM_FIRES]
end

global AGG_PREC = 30
global PASSIVE_STATES = 5

ts["ws"] = @elapsed ws, ws_dict = warm_start_suppression_plans(10, fire_configs, r_data, c_data, rotation_order, arc_costs, 
                                                     gamma, false, false, 3600)

ws_duals = dual.(ws_dict["f_linking"])
int_aware_duals = ws_duals * 0


if params["apply_warm_start"]
    for fire in 1:NUM_FIRES
        ws_fire = ws[fire]
        for plan in ws_fire
            cost = get_fire_cost(plan, fire_configs[fire])
            update_available_supp_plans(fire, cost, plan, col_gen_data.suppression_plans)
        end
    end
end


global AGG_PREC = params["agg_prec"]
global PASSIVE_STATES =  params["passive_states"]


current_num_routes = copy(col_gen_data.routes.routes_per_crew)
current_num_plans = copy(col_gen_data.suppression_plans.plans_per_fire)

objs = []
max_iters = params["max_iters"]
n_iters = 0
opt = false
ts["cg"] = 0
max_time = params["cg_time_limit"]
col_gen_config = Dict{String,Any}("ws_dual" => false)
col_gen_config["master_problem"] = Dict{String,Any}()

col_gen_config["master_problem"]["dual_stabilization"] = params["dual_stab_type"]
col_gen_config["master_problem"]["dual_secondary_eps"] = params["dual_stab_eps"]
col_gen_config["master_problem"]["dual_warm_start"] = ws_duals
if params["dual_stab_anchor"] == "all_ones"
    col_gen_config["master_problem"]["dual_warm_start"] = ones(size(col_gen_config["master_problem"]["dual_warm_start"]))
end

In [112]:
mp = master_problem(col_gen_config, col_gen_data.routes, col_gen_data.suppression_plans, 
                    r_data, rotation_order, 0, false);

In [113]:
models2 = []

while n_iters < 50
    push!(models2, deepcopy(mp))
    n_iters += 1
    dual_stab_weight = params["ws_dual_weight"][n_iters] + params["int_aware_dual_weight"][n_iters]
    col_gen_config["ws_dual_weight"] = dual_stab_weight
    col_gen_config["ws_dual"] = params["ws_dual_weight"][n_iters] * ws_duals
    if dual_stab_weight > 0
        col_gen_config["ws_dual"] = col_gen_config["ws_dual"] / dual_stab_weight
    end

    for fire in 1:NUM_FIRES
        fire_solver_configs[fire]["int_aware_adjustment_pattern"] = params["int_aware_price_adjustment"][n_iters]

        # no int-aware adjustment
        fire_solver_configs[fire]["int_aware_adjustment_pattern"] = [(0, 0)]
    end

    last_num_routes = copy(col_gen_data.routes.routes_per_crew)
    last_num_plans = copy(col_gen_data.suppression_plans.plans_per_fire)



    ts["cg"] += @elapsed mp, a, b, c = run_CG_step(col_gen_data, A, arc_costs, g_data, r_data, fire_configs, 
                                          fire_solver_configs, col_gen_config, rotation_order, gamma,  
                                          params["restore_cost"], mp)

    push!(objs, a)
    push!(rhos, b)
    push!(allotment_history, c)
end



solve
0.0059503
formulate
0.0002928
solve
0.0083367
formulate
0.00138
solve
0.0077321
formulate
0.0013313
solve
0.0058867
formulate
0.0014403
solve
0.0096581
formulate
0.0017504999999999999
solve
0.0070498
formulate
0.0016098
solve
0.0074116
formulate
0.0014444
solve
0.0086142
formulate
0.0014015
solve
0.0135261
formulate
0.0014346
solve
0.0128624
formulate
0.001419
solve
0.0146614
formulate
0.0013922
solve
0.0134709
formulate
0.0014593
solve
0.0109402
formulate
0.0013790999999999999
solve
0.0109005
formulate
0.0014399
solve
0.0104213
formulate
0.0014054
solve
0.0078226
formulate
0.0014077
solve
0.0096743
formulate
0.0016017000000000002
solve
0.0146378
formulate
0.0020038
solve
0.0155088
formulate
0.0018457999999999999
solve
0.0139724
formulate
0.0014391
solve
0.0103284
formulate
0.0014674
solve
0.0105549
formulate
0.0014986
solve
0.0096624
formulate
0.0016885
solve
0.0103873
formulate
0.0014533
solve
0.0086312
formulate
0.0014586
solve
0.0078118
formulate
0.0014253
solve
0.0051438
for

In [117]:
objective_value(models[49]["m"])

2.670311016847217e6

In [118]:
allotment_history[end]

6×14 Matrix{Float64}:
 0.0       0.0         0.0        …  16.866      1.65694   0.0  0.0
 0.0       0.0130814   0.0130814      0.0        0.0       0.0  0.0
 9.0       1.99496     0.0130814      0.0        0.0       0.0  0.0
 5.50989  18.0312     19.3688         0.0        0.368824  0.0  0.0
 0.0       0.0         0.0            0.0670716  1.0       0.0  0.0
 5.46935   0.0130814   0.617338   …   0.0392442  0.617338  0.0  0.0

In [114]:
optimize!(models2[49]["m"])
objective_value(models2[49]["m"])

2.6703110168472184e6

### Run iteration

In [122]:

n_iters += 1
dual_stab_weight = params["ws_dual_weight"][n_iters] + params["int_aware_dual_weight"][n_iters]
col_gen_config["ws_dual_weight"] = dual_stab_weight
col_gen_config["ws_dual"] = params["ws_dual_weight"][n_iters] * ws_duals
if dual_stab_weight > 0
    col_gen_config["ws_dual"] = col_gen_config["ws_dual"] / dual_stab_weight
end

for fire in 1:NUM_FIRES
    fire_solver_configs[fire]["int_aware_adjustment_pattern"] = params["int_aware_price_adjustment"][n_iters]

    # no int-aware adjustment
    fire_solver_configs[fire]["int_aware_adjustment_pattern"] = [(0, 0)]
end

last_num_routes = copy(col_gen_data.routes.routes_per_crew)
last_num_plans = copy(col_gen_data.suppression_plans.plans_per_fire)

if n_iters == 100000
    ts["cg"] += @elapsed mp = run_CG_step(col_gen_data, A, arc_costs, g_data, r_data, fire_configs, 
                                          fire_solver_configs, col_gen_config, rotation_order, gamma,  
                                          params["restore_cost"])
        push!(objs, objective_value(mp["m"]))
        push!(rhos, dual.(mp["rho"]))
        push!(allotment_history, get_fire_allotments(mp, col_gen_data))

else

    ts["cg"] += @elapsed mp, a, b, c = run_CG_step(col_gen_data, A, arc_costs, g_data, r_data, fire_configs, 
                                          fire_solver_configs, col_gen_config, rotation_order, gamma,  
                                          params["restore_cost"], mp)
    
    push!(objs, a)
    push!(rhos, b)
    push!(allotment_history, c)
end;


solve
0.0001215
formulate
4.0e-6


### Get number of plans per fire

In [1315]:
n_iters

35

In [1316]:
close_iters = [i for i in 1:n_iters if objs[i] / last(objs) < 1.05];

In [1317]:
col_gen_data.suppression_plans.plans_per_fire

6-element Vector{Int64}:
 28
 30
  3
 30
 35
 21

In [1319]:
col_gen_data.suppression_plans.plans_per_fire

6-element Vector{Int64}:
  52
 123
   3
  80
 168
  30

In [1225]:
using StatsBase

In [1318]:
offsets = [-2, -1, 0, 1, 2]

for fire in 1:NUM_FIRES
    
    plans = []
    
    for offset in offsets
        existing_plans = [col_gen_data.suppression_plans.crews_present[fire, i, :] 
                          for i in 1:col_gen_data.suppression_plans.plans_per_fire[fire]]

        sp_config = copy(fire_solver_configs[fire])
        sp_config["model_data"] = fire_configs[fire]
        sp_config["solver_strategy"] = "ceiling"
        sp_config["warm_start"] = false
        sp_config["capacities"] = max.(0, last(allotment_history)[fire, :] .+ offset)

        sp_config["break_capacity_penalty"] = 0

        mean_rho = sum(last(rhos)[fire, :]) / length(last(rhos)[fire, :])

        plans = []

        for lambda in [0.8, 0.9, 1, 1.1, 1.2]
            
            ix = sample(close_iters)
            
            cost, rel_cost, plan =
            run_fire_subproblem(col_gen_data.plan_sps[fire], sp_config, lambda * rhos[ix][fire, :])

            cap_pens = exp.(LinRange(1, log(rel_cost), 25)) .- exp(1)
            cap_pens[1] = 0


            for cap_pen in cap_pens
                sp_config["break_capacity_penalty"] = cap_pen
                cost, rel_cost, plan = run_fire_subproblem(col_gen_data.plan_sps[fire], sp_config, lambda * rhos[ix][fire, :])
                if ~(plan in plans) & ~(plan in existing_plans)
                    update_available_supp_plans(fire, cost, plan, col_gen_data.suppression_plans)
                    push!(plans, plan)
                end
            end
        end
    end
end

In [1321]:
ts

Dict{String, Float64} with 5 entries:
  "init_cg"                    => 0.752721
  "price_and_branch"           => 5.9677
  "price_and_branch_formulate" => 1.0134
  "cg"                         => 19.7658
  "ws"                         => 3.94017

In [1329]:
tmp = deepcopy(col_gen_config["master_problem"]["dual_stabilization"])
col_gen_config["master_problem"]["dual_stabilization"] = false
ts["price_and_branch_formulate"] = @elapsed pb = master_problem(col_gen_config, col_gen_data.routes, col_gen_data.suppression_plans, 
                    r_data, rotation_order, gamma, true)
ts["price_and_branch"] = @elapsed optimize!(pb["m"])
col_gen_config["master_problem"]["dual_stabilization"] = tmp
convert.(Int, round.(100 .* get_fire_allotments(pb, col_gen_data)) ./ 100)

6×14 Matrix{Int64}:
  0   0   0   0  0  0  0  0   0   0   2  12  2  0
  0   0   0   0  4  4  8  8  10   0   0   0  0  0
 10   0   0   0  0  0  0  0   0   0   0   0  0  0
  1  20  20  14  2  0  0  0   0   0   4   0  0  0
  0   0   0   0  6  6  6  6   6  11  11   6  0  0
  9   0   0   0  0  0  0  0   0   0   0   0  0  0

In [1333]:
[i for i in eachindex(pb["plan"]) if value(pb["plan"][i]) > 0.001]

6-element Vector{Tuple{Int64, Int64}}:
 (1, 52)
 (2, 95)
 (6, 2)
 (5, 122)
 (4, 49)
 (3, 2)

In [1304]:
ts

Dict{String, Float64} with 5 entries:
  "init_cg"                    => 3.06647
  "price_and_branch"           => 303.79
  "price_and_branch_formulate" => 4.18953
  "cg"                         => 185.12
  "ws"                         => 9.22496

In [1330]:
pb["m"]

A JuMP Model
Minimization problem with:
Variables: 1409
Objective function type: AffExpr
`AffExpr`-in-`MathOptInterface.EqualTo{Float64}`: 20 constraints
`AffExpr`-in-`MathOptInterface.GreaterThan{Float64}`: 390 constraints
`VariableRef`-in-`MathOptInterface.GreaterThan{Float64}`: 1409 constraints
`VariableRef`-in-`MathOptInterface.Integer`: 1409 constraints
Model mode: AUTOMATIC
CachingOptimizer state: ATTACHED_OPTIMIZER
Solver name: Gurobi
Names registered in the model: linking, out_of_region, plan, plan_per_fire, q, route, route_per_crew

In [1331]:
objective_value(pb["m"])

3.0399192719967533e6

In [1265]:
col_gen_data.suppression_plans.crews_present[1, 80:100, :]

21×14 Matrix{Int8}:
 0  0  0  0   1  2  8  7  7   9   3  0  0  0
 0  0  0  0   1  5  6  7  7   5   1  0  0  0
 0  0  0  0   1  5  7  6  7   5   6  0  0  0
 0  0  0  0   1  0  1  4  7   8   8  0  0  0
 0  0  0  0   1  0  3  7  6   6   8  0  0  0
 0  0  0  0   1  5  6  7  7   5   6  0  0  0
 0  0  0  0   1  5  0  2  0  10  10  0  0  0
 0  0  0  0   1  5  0  4  5  10   7  0  0  0
 0  0  0  0   1  5  0  6  6   9   6  0  0  0
 0  0  0  0   1  5  0  6  6   8   5  1  0  0
 0  0  0  0   1  5  0  7  7   5   3  0  0  0
 0  0  0  0   1  5  0  7  7   5   1  0  0  0
 0  9  9  5  10  6  9  7  8   0   0  0  0  0
 0  9  9  0   2  6  8  7  7   6   0  0  0  0
 0  3  2  0   2  7  8  8  8   6   2  0  0  0
 0  0  0  0   3  6  8  8  8   6   3  0  0  0
 0  0  0  0   1  5  8  8  8   6   2  0  0  0
 0  9  9  0   2  6  8  9  8   4   0  0  0  0
 0  0  0  0   3  6  8  8  8   7   2  0  0  0
 0  0  0  0   1  5  8  7  7   6   2  0  0  0
 0  0  0  0   1  5  8  7  7   6   3  1  0  0

## Stabilized

In [571]:
params = Dict{String, Any}()
params["gamma"] = 0

params["in_path"] = "data/raw/big_fire"
params["agg_prec"] = 10
params["passive_states"] = 30

params["agg_prec"] = 30
params["passive_states"] = 5

params["num_fires"] = 6
params["num_crews"] = 20
params["fire_solver_type"] = "dp"

params["apply_warm_start"] = false
params["restore_cost"] = false

params["solve_explicit_lin_time_limit"] = 60
params["solve_explicit_int_time_limit"] = 0
params["cg_time_limit"] = 240
params["solve_net_flow_time_limit"] = 360

params["max_iters"] = 1000

params["dual_stab_type"] = "global"
params["dual_stab_anchor"] = false
params["dual_stab_eps"] = 0.01

params["ws_dual_weight"] = zeros(params["max_iters"])
params["int_aware_dual_weight"] = zeros(params["max_iters"])

start_adj = 1000
pattern = [(0, 1e30), (1, 1e30), (2, 1e30), (0, 0)]
params["int_aware_price_adjustment"] = [i <= start_adj ? [(0, 0)] : pattern for i in 1:params["max_iters"]]

d2, cg_data2 = run_tests(params);

In [9]:
function get_out_of_region_stats(region, arcs_used, region_data)
    """
    """
    
    # restrict to the arcs that exited the given region
    out_of_region_ixs = [i for i in length(arcs_used[:, 1]) if arcs_used[i, 10] == region]
    out_of_region_arcs = arcs_used[out_of_region_ixs, :]
    
    # get the crews associated with this region
    crews = [i for i in 1:length(region_data.crew_regions) if region_data.crew_regions[i] == region]
    
    # initialize output array of indicator variables for crews exiting region
    out_array = zeros(length(crews), NUM_TIME_PERIODS)
    
    # for each crew in the region
    for i in 1:length(crews)
        
        # restrict to the arcs involving this crew
        crew_ixs = [j for j in length(out_of_region_arcs[:, 1]) if arcs_used[j, 1] == crews[i]]
        crew_arcs = out_of_region_arcs[crew_ixs, :]
        
        # get the times of rotation
        rotation_times = [t in crew_arcs[:, 6] ? 1 : 0 for t in 1:NUM_TIME_PERIODS]
        
        # update output for crew
        out_array[i, :] = rotation_times
        
    end
    
    return out_array
end

LoadError: syntax: incomplete: "function" at In[9]:1 requires end